In [ ]:
outdir = '202011'
product_name = '贝达'

In [1]:
# 产品匹配表

import pandas as pd
df = pd.read_excel(u"/home/ywyuan/max_project/prod_mapping.xlsx", dtype="object")
print(df.head(4))

       ID         PHA Province City
0  230461  PHA0000591      安徽省  蚌埠市
1  230011  PHA0005209      安徽省  合肥市
2  230021  PHA0005207      安徽省  合肥市
3  230031  PHA0005208      安徽省  合肥市


In [2]:
df[df.columns] = df[df.columns].astype("str")
  
df_spark = spark.createDataFrame(df)

for each in df_spark.columns:
  df_spark = df_spark.withColumn(each, func.when(func.col(each) == 'nan', func.lit(None)).otherwise(func.col(each)))
df_spark.show()

NameError: name 'spark' is not defined

In [ ]:
df_spark = df_spark.distinct().repartition(1)
df_spark.write.format("parquet") \
    .mode("overwrite").save("s3a://ph-max-auto/v0.0.1-2020-06-08/" + product_name + "/" + outdir + "/prod_mapping")
#df_spark.write.format("parquet") \
#    .mode("overwrite").save("s3a://ph-max-auto/v0.0.1-2020-06-08/" + product_name + "/province_city_mapping")

In [ ]:
#  =========  max_result_path_list 调整  =============

max_result_path_list = spark.read.csv("s3a://ph-max-auto/v0.0.1-2020-06-08/" + product_name + "/max_result_path_list.csv", header=True)
max_result_path_list.collect()

In [ ]:
list_dict = [
  {"project":"贝达","path":"s3a://ph-max-auto/v0.0.1-2020-06-08/贝达/202006/MAX_result/MAX_result_201701_202006_city_level", 
  "time_left":"201701", "time_right":"201912"},
  {"project":"贝达","path":"s3a://ph-max-auto/v0.0.1-2020-06-08/贝达/202011/MAX_result/MAX_result_202001_202011_city_level", 
  "time_left":"202001", "time_right":"202011"}
]

In [ ]:
max_result_path_list_new = spark.createDataFrame(list_dict)
max_result_path_list_new = max_result_path_list_new.select("project", "path", "time_left", "time_right")
max_result_path_list_new.show()
max_result_path_list_new.toPandas()["path"].values

In [ ]:
max_result_path_list_new = max_result_path_list_new.repartition(1)
max_result_path_list_new.write.format("csv").option("header", "true") \
    .mode("overwrite").save("s3a://ph-max-auto/v0.0.1-2020-06-08/" + product_name + "/max_result_path_list.csv")

In [ ]:
# load